# 0.1 Imports

In [1]:
import pandas as pd
import numpy as np
import os

# 0.2 Funções Auxiliares

In [2]:
# Checking all columns values from a given data frame
def checkColumnsValues(dataFrame):
    col_vals = {}
    for i in dataFrame.columns:
        sorted = dataFrame[i].unique().tolist()
        #sorted.sort()
        col_vals[i] = [sorted]
    return pd.DataFrame(col_vals)

In [3]:
def create_word_dict(arr):
    word_dict = {}
    for i in arr:
        if len(i.split()) not in word_dict:
            word_dict[len(i.split())] = []
        word_dict[len(i.split())].append(i)
    
    return word_dict

# 0.3 Load csvs

In [4]:
crimes_1 = pd.read_csv('D:/Universidade-Fcul/2_semestre/TPD/projeto/BoroughCrimes.csv')
crimes_2 = pd.read_csv('D:/Universidade-Fcul/2_semestre/TPD/projeto/MPS Borough Level Crime.csv')
demographics = pd.read_csv('D:/Universidade-Fcul/2_semestre/TPD/projeto/demographics.csv')

In [5]:
print('crimes1 shape:', crimes_1.shape[0])
print('crimes2 shape:', crimes_2.shape[0])

crimes1 shape: 3861000
crimes2 shape: 1052


# 1. Comparar crime Dataframes

In [6]:
# Major Categorys
crimes_1_Mc = crimes_1['major_category'].sort_values(ascending=True).unique().tolist()
crimes_2_Mc = crimes_2['Major Category'].sort_values(ascending=True).unique().tolist()
print('Major Categorys are equal? ',crimes_1_Mc == crimes_2_Mc)

# Minor Categorys
crimes_1_mc = crimes_1['minor_category'].sort_values(ascending=True).unique().tolist()
crimes_2_mc = crimes_2['Minor Category'].sort_values(ascending=True).unique().tolist()
print('Minor Categorys are equal? ',crimes_1_mc == crimes_2_mc)

# Boroughs
crimes_1_b = crimes_1['borough'].sort_values(ascending=True).unique().tolist()
crimes_2_b = crimes_2['Borough'].sort_values(ascending=True).unique().tolist()
print('Boroughs are equal? ',crimes_1_b == crimes_2_b)

Major Categorys are equal?  True
Minor Categorys are equal?  False
Boroughs are equal?  False


As major categories coincidem em ambos os dataframes, mas os bairros e as minor caregorys precisão de uma analise em mais detalhe.

### 1.1 Comparar Boroughs

In [7]:
crime_1_words = create_word_dict(crimes_1_b)
crime_2_words = create_word_dict(crimes_2_b)

In [20]:
equal = []
for i in crime_1_words[1]:
    equal.append(i in crime_2_words[1])

print('Are crimes 1 boroughs (len = 1) in crimes 2 boroughs? ', equal)

equal = []
for i in crime_1_words[3]:
    equal.append(i in crime_2_words[3])
    
print('Are crimes 1 boroughs (len = 3) in crimes 2 boroughs? ', equal)

Are crimes 1 boroughs (len = 1) in crimes 2 boroughs?  [True, True, True, True, True, True, True]
Are crimes 1 boroughs (len = 3) in crimes 2 boroughs?  [False, True, True]


1. Todos os boroughs de crimes 1 estão escritos da mesma maneira no boroughs 2.
2. Comparando as palavras de tamanho 3, vimos que a unica cidade diferente é 'City of london' e visto que vamos remover esta cidade nos passos futuros devido a analise do dataframe 'demographics' isto nao representa um problema.

### 1.2 Comparar Minor Categorys

In [9]:
crime_1_words = create_word_dict(crimes_1_mc)
crime_2_words = create_word_dict(crimes_2_mc)

In [10]:
# print(crime_1_words)
# print(crime_2_words)

In [11]:
for n in range(1,6):
    equal = []
    for i in crime_1_words[1]:
        equal.append(i in crime_2_words[1])
    print('Minor Category Size {size}, {e}'.format(size = n, e = equal))

Minor Category Size 1, [True, True, True, True]
Minor Category Size 2, [True, True, True, True]
Minor Category Size 3, [True, True, True, True]
Minor Category Size 4, [True, True, True, True]
Minor Category Size 5, [True, True, True, True]


Olhando para as minor categoriesm conseguimos ver todas elas estão escritas da mesma  maneira. Podemos então proceguir com o merge destes dataframes.

#### Transform crimes_2 into crimes_1 format, for merging

In [12]:
crimes_1.head()

,lsoa_code,borough,major_category,minor_category,value,year,month
0,E01000002,City of London,Violence Against the Person,Harassment,0,2011,9
1,E01000005,City of London,Violence Against the Person,Harassment,0,2014,7
2,E01000001,City of London,Violence Against the Person,Harassment,0,2008,3
3,E01032739,City of London,Violence Against the Person,Harassment,0,2016,3
4,E01000001,City of London,Violence Against the Person,Harassment,0,2011,10


In [13]:
crimes_2.head()

,Borough,Major Category,Minor Category,201609,201610,201611,201612,201701,201702,201703,...,201711,201712,201801,201802,201803,201804,201805,201806,201807,201808
0,Barking and Dagenham,Burglary,Burglary - Business and Community,45,41,24,19,25,47,44,...,27,21,38,33,38,36,24,33,30,19
1,Barking and Dagenham,Burglary,Burglary - Residential,49,60,73,100,118,124,93,...,88,124,143,134,122,75,93,80,93,84
2,Barking and Dagenham,Criminal Damage,Arson,15,3,1,5,5,5,2,...,7,4,2,3,6,3,4,12,6,5
3,Barking and Dagenham,Criminal Damage,Criminal Damage To M/V,61,68,67,59,65,62,61,...,48,57,60,51,53,64,64,66,69,43
4,Barking and Dagenham,Criminal Damage,Criminal Damage To Other Bldg,11,12,16,8,10,6,13,...,11,10,10,5,6,9,9,8,7,7


In [14]:
# Arranjar colunas do dataframe 2
crimes_2.columns.tolist()

# Arranjar apenas as colunas dos anos
years_months = crimes_2.columns.tolist()[3:]

# Inicializar o array das linhas 
rows = []

for i,r in crimes_2.iterrows(): 
    
    # Template para cada linha do crimes_2
    template = [r['Borough'], r['Major Category'], r['Minor Category']]
    
    # Arranjar value, year and month
    for ym in years_months:
        row = template.copy()
        row.extend((r[ym],ym[:4],ym[4:]))
        rows.append(row)

In [15]:
# Arranjar colunas para o dataframe (não sei pq mas nao tava a conseguir fazer isso tudo numa linha)
cols = crimes_1.columns.tolist()
cols.remove('lsoa_code')

crimes_3 = pd.DataFrame(data=rows, columns = cols)
print(crimes_3.shape)
crimes_3.head()

(25248, 6)


,borough,major_category,minor_category,value,year,month
0,Barking and Dagenham,Burglary,Burglary - Business and Community,45,2016,09
1,Barking and Dagenham,Burglary,Burglary - Business and Community,41,2016,10
2,Barking and Dagenham,Burglary,Burglary - Business and Community,24,2016,11
3,Barking and Dagenham,Burglary,Burglary - Business and Community,19,2016,12
4,Barking and Dagenham,Burglary,Burglary - Business and Community,25,2017,01


#### Check for comon years

In [16]:
print('Crimes 1: ', np.sort(crimes_1['year'].unique()))
print('Crimes 2: ', np.sort(crimes_3['year'].unique()))

Crimes 1:  [2008 2009 2010 2011 2012 2013 2014 2015 2016]
Crimes 2:  ['2016' '2017' '2018']


Vimos que 2016 está em ambos os datasets. É preciso então ter em conta o este ano como potencial zona de criação de duplicados.

#### Merge both datasets

In [17]:
crimes_merged = pd.concat([crimes_1,crimes_3])
print(crimes_merged.shape)
crimes_merged.head()

(3886248, 7)


,lsoa_code,borough,major_category,minor_category,value,year,month
0,E01000002,City of London,Violence Against the Person,Harassment,0,2011,9
1,E01000005,City of London,Violence Against the Person,Harassment,0,2014,7
2,E01000001,City of London,Violence Against the Person,Harassment,0,2008,3
3,E01032739,City of London,Violence Against the Person,Harassment,0,2016,3
4,E01000001,City of London,Violence Against the Person,Harassment,0,2011,10


In [18]:
# Drop duplicates
crimes_merged = crimes_merged.drop_duplicates()

# Drop lsoa_code
crimes_merged = crimes_merged.drop(['lsoa_code'],axis=1)
print(crimes_merged.shape)

(3886248, 6)


Verificamos os duplicados antes de remover o lsoa_code, para não estarmos a identificar linhas como falsos duplicados, depois de de remover esta coluna.

No duplicates where found

#### Export crimes_merged to csv

In [19]:
# compression_opts = dict(method='zip',
#                         archive_name='crimes_merged.csv') 

# crimes_merged.to_csv('out.zip', 
#                     compression=compression_opts,
#                     index=False)